## Проверка сохранения модели

Построим тестовый ноутук для выбора оптимального пути сохранения и загрузки модели.

При использовании `pytorch` пока найдено два способа:
- `save_pretrained`
- `save_dict` - он сложнее, но есть шанс, что будет меньше места занимать

In [17]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, MarianMTModel

DEFAULT_FILEPATH = "../../opus-mt-en-ru"

tokenizer = AutoTokenizer.from_pretrained(DEFAULT_FILEPATH)

model = AutoModelForSeq2SeqLM.from_pretrained(DEFAULT_FILEPATH)

Some weights of MarianMTModel were not initialized from the model checkpoint at ../../opus-mt-en-ru and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
input_ids = tokenizer("My name is Wolfgang and I live in Berlin", return_tensors='pt').input_ids

decoder_input_ids = tokenizer("Меня зовут Вольфганг и я живу в Берлине.", return_tensors='pt').input_ids

output = model(input_ids=input_ids, labels=decoder_input_ids)
print(output[0])

tensor(4.8305, grad_fn=<NllLossBackward>)


In [19]:
store_path = "./store-pretrained"
tokenizer.save_pretrained(store_path)
model.save_pretrained(store_path)

In [ ]:
from pathlib import Path

root_directory = Path('.')
sum(f.stat().st_size for f in root_directory.glob('**/*') if f.is_file())

In [10]:
from pathlib import Path

root_directory = Path(store_path)
size_save_pretrained = sum(f.stat().st_size for f in root_directory.glob('**/*') if f.is_file())

size_save_pretrained /= 2 ** 20

root_directory = Path(DEFAULT_FILEPATH)
size_original = sum(f.stat().st_size for f in root_directory.glob('**/*') if f.is_file())

size_original /= 2 ** 20

In [13]:
print("Storage memory of original model from HFace {:g} MB and stroed model by save_pretrained {:g} MB.".format(size_original, size_save_pretrained))

Storage memory of original model from HFace 1665.45 MB and stroed model by save_pretrained 295.2 MB.


In [20]:
loaded_tokenizer = AutoTokenizer.from_pretrained(store_path)
loaded_model = AutoModelForSeq2SeqLM.from_pretrained(store_path)

In [21]:
input_ids = loaded_tokenizer("My name is Wolfgang and I live in Berlin", return_tensors='pt').input_ids

decoder_input_ids = loaded_tokenizer("Меня зовут Вольфганг и я живу в Берлине.", return_tensors='pt').input_ids

output = loaded_model(input_ids=input_ids, labels=decoder_input_ids)
print(output[0])

tensor(4.8305, grad_fn=<NllLossBackward>)
